In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

## Загрузка данных:

In [3]:
# Глобальные переменные 

dollar_exchange_rate = 70
euro_exchange_rate = 70  

                            #ГГ  ММ ДД
date_of_quarter_begin = date(2023, 1, 1)
date_of_quarter_end = date(2023, 3, 31)

name_for_table = 'Таблица выполнения плана.xlsx'
name_for_file_output = 'Файл расчета выполнения плана ЮФО.xlsx'

# названия файлов загрузки
path_data_with_deals = r"C:\Users\kakorin_is\Desktop\JupiterNotebook_files\Дела ЮФО на 28.02.xlsx"
path_activities =  r'C:\Users\kakorin_is\Desktop\JupiterNotebook_files\активности нст на 15 февр.xlsx'
path_earliers_doned = r'C:\Users\kakorin_is\Desktop\Свод\Перечень учтенных ранее ПА (с учетом 3 квартала 2022 года).xlsx'

In [4]:
dataset = pd.read_excel(path_data_with_deals, index_col=None)

earliers_doned = pd.read_excel(path_earliers_doned, index_col=None)

activities = pd.read_excel(path_activities, index_col=None)


In [5]:
dataset.shape

(5555, 60)

In [6]:
# cмотрим список колонок стандартной выгрузки из БД (не все будут нужны)
dataset.columns

Index(['Номер по SIEBEL', 'Ответственный', 'Договор', 'Заемщик ФЛ',
       'Заемщик КМБ', 'Тип', 'РОО', 'Стадия', 'Сумма заявленных требований',
       'Номер дела в суде (1-ая инстанция)', 'Суд', 'Передача обращений в суд',
       'Критерий нестандартности', 'Дата судебного решения',
       'Дата последнего судебного заседания',
       'Результат последнего судебного заседания', 'Тип последней активности',
       'Дата начала последней активности', 'Сумма взыскания по решению суда',
       'Комментарии', 'Дата создания дела', 'Результат', 'Дата закрытия',
       'Статус', 'Нормативный срок получения ИД', 'Дело на контроле',
       'Сайт суда', 'Общая сумма ПЗ', 'Общая сумма ПЗ по ОД',
       'Сумма просроченных процентов', 'Сумма просроченных комиссий',
       'Передано на аутсорсинг/внешние консультанты',
       'Организация/ФИО привлеченного ВК', 'Куратор/Координатор по делу',
       'Пени', 'Продукт', 'Создатель', 'Передано в ЮУ/ЮСТП', 'Дата передачи',
       'ЦС подсудности', 'ТП 

In [7]:
# так он выглядит (для примера вывожу обесзличенные столбцы)
dataset.loc[:, ['Номер по SIEBEL','Тип','Дата создания дела','Дата закрытия', ]]

,Номер по SIEBEL,Тип,Дата создания дела,Дата закрытия
0,1-2946161483,Банкротство ФЛ РБ,2022-10-21 08:55:46,2022-12-02
1,1-2922685695,Взыскание задолженности ПА РБ,2022-10-11 11:12:52,NaT
2,1-2777993227,Банкротство ФЛ РБ,2022-08-24 15:11:39,2022-10-05
3,1-2829411327,Банкротство ФЛ РБ,2022-09-09 15:08:09,2022-11-17
4,1-2562329796,Банкротство ФЛ РБ,2022-06-03 15:10:32,2022-08-24
...,...,...,...,...
5550,1-1533535923,Банкротство ФЛ РБ,2021-05-24 14:14:21,2021-08-13
5551,1-1643646792,Банкротство ФЛ РБ,2021-06-29 13:32:18,2022-01-10
5552,1-1640798290,Взыскание задолженности ПА МБ,2021-06-28 11:50:58,2021-07-06
5553,1-1619257247,Банкротство ФЛ РБ,2021-06-16 15:02:39,2022-01-10


In [8]:
data = dataset.copy()

## Описание алгоритма:

- Преобразовать поле Дата создания дела в поле с только с датой (убрать время).
<br>

- Удалить дела закрытые до отчетного квартала и завершенные в этом квартале результатами Возврат, Отказано, а также отмененые дела.
<br>
- Подтянуть по номеру по сибель и договору ранее учтенные дела, чтобы потом исключить их из СМОТ
<br>
- Подтянуть по номеру по сибель активность НСТ из выгрузки нестандартных активностей, чтобы разметить стандартные и нест. дела (разные нормативные сроки у них)
<br>
- Коррекция сумм: <br>
    а) убрать символы  руб. из Заявл. требований и Суммы взысканного по решению суда.<br>
    б) убрать символы доллара и евро из Заявл. требований и при этом перевести суммы из валюты в рубли.<br>
    в) по полю Валюта найти валютные кд и преобразовать колонки Сумма ОД и % по ним,
    
 <br>
 
- Коррекция колонки Тип (выделение Банкротов РБ).
- Создание колонки Дата конца (имеется ввиду дата конца квартала - будем из нее вычитать).
- Создание колонок: Кол-во дней, Коэф,  Факт, План.
<br>
- Заполняю колонки по не закрытым делам:<br>
    а) Кол-во дней - вычитаю из даты конца дату начала,<br>
    г) Активность НСТ - присоединяю данные о наличии в карточке дела активности НСТ,<br>
    д) Стандарт/нестандарт - заполняю, исходя из наличия активности НСТ в карточке дела смотрим поле Активность НСТ.<br>
    е) Коэф - исходя из значения поля Стандарт/нестандарт и кол-ва дней в поле Кол-во дней проставляется соотв. коэф.<br>
    ж) Факт - исходя из проставленного коэф заполняются путем умножения к. на сумму ОД и % по кд либо умножения на Заявл. требования, если дело банкротное.<br>
    з) План - заполняется по аналогии с Фактом.

__Правим поле Дата создания дела:__

In [9]:
# Функция преобразования даты
def fix_date(some_date):
    some_date = some_date.date()
    
    return some_date

# Специально делаю отдельный столбец к корректной формой даты, чтобы можно было потом сравнить
data['Дата создания'] = data['Дата создания дела'].apply(lambda x: fix_date(x))

In [10]:
# Проверим корректно ли произошли преобразования
data.loc[:, ['Дата создания дела', 'Дата создания']]

,Дата создания дела,Дата создания
0,2022-10-21 08:55:46,2022-10-21
1,2022-10-11 11:12:52,2022-10-11
2,2022-08-24 15:11:39,2022-08-24
3,2022-09-09 15:08:09,2022-09-09
4,2022-06-03 15:10:32,2022-06-03
...,...,...
5550,2021-05-24 14:14:21,2021-05-24
5551,2021-06-29 13:32:18,2021-06-29
5552,2021-06-28 11:50:58,2021-06-28
5553,2021-06-16 15:02:39,2021-06-16


## Удаляем дела 

Поскольку выгрузка из БД содержит все вообще дела за все время, то нужно оставить только активные дела и завершенные в этом квартале.

In [11]:
# правим тип данных
data['Дата закрытия'] = data['Дата закрытия'].apply(lambda x: fix_date(x))

In [12]:
to_del_old = data.loc[(data['Статус'] == 'Завершено') & (date_of_quarter_begin > data['Дата закрытия'])]
to_del_old.index                                        #datetime(2023, 1, 1, 0, 0, 0, 1).date()

Int64Index([   0,    2,    3,    4,    6,    8,    9,   10,   11,   12,
            ...
            5545, 5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554],
           dtype='int64', length=4238)

In [13]:
# удаляю все завершенное до 01.01.2023
data = data.drop(index=to_del_old.index)
data.shape

(1317, 61)

In [14]:
data['Статус'].value_counts()

В работе             843
Завершено            416
Отменено              49
Приостановлено         5
Частично передано      4
Name: Статус, dtype: int64

In [15]:
data = data.loc[data['Статус'] != 'Отменено']
data.shape

(1268, 61)

In [16]:
# Смотрим какие Результаты имеются в оставленных делах
data.loc[data['Статус'] == 'Завершено']['Результат'].value_counts()

Включено в РТК                    320
ИД передан в УПК                   55
Включено за РТК                    15
Возврат в досудебное пр-во         15
Получен судебный акт                3
Создано ошибочно                    3
Отказано во вкл в РТК               3
Прекращение дела о Банкротстве      1
Name: Результат, dtype: int64

In [17]:
# Из выгрузки нужно удалить Возвращенное, Ошибочное и Отказные дела

to_del_2 = data.loc[(data['Статус'] == 'Завершено') & ((data['Результат'] == 'Возврат в досудебное пр-во') | \
                                                       (data['Результат'] == 'Создано ошибочно') | \
                                                       (data['Результат'] == 'Отказано во вкл в РТК') | \
                                                       (data['Результат'] == 'Прекращение дела о Банкротстве') | \
                                                       (data['Результат'] == 'Отказано в рассмотрении'))]
to_del_2 ['Результат'].value_counts()

Возврат в досудебное пр-во        15
Отказано во вкл в РТК              3
Создано ошибочно                   3
Прекращение дела о Банкротстве     1
Name: Результат, dtype: int64

In [18]:
data = data.drop(index=to_del_2 .index)
data.shape

(1246, 61)

# Подтягиваю ранее учтенные

In [19]:
# смотрим какие колонки в таблице и ее размер
earliers_doned.columns, earliers_doned.shape

(Index(['Номер по SIEBEL', 'Договор', 'Квартал учета в факте'], dtype='object'),
 (25275, 3))

In [20]:
earliers_doned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25275 entries, 0 to 25274
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Номер по SIEBEL        25275 non-null  object
 1   Договор                14342 non-null  object
 2   Квартал учета в факте  25275 non-null  object
dtypes: object(3)
memory usage: 592.5+ KB


В таблице earliers_doned содержится список кд и номеров по Сибель дел, которые ранее учитывались в СМОТ и их нужно исключать.<br>

Также видим, что в некоторых случаях отдельно зафиксированы номера кд по учтенным делам, а отдельно номера по Сибель - будем учитывать это при подтягивании значений

In [21]:
earliers_doned

,Номер по SIEBEL,Договор,Квартал учета в факте
0,1-1609520,014/2011-5-69,1 квартал 2019
1,1-1609534,634КЛ/13,1 квартал 2019
2,1-1609543,007/2015-0/3-69,1 квартал 2019
3,1-1263113,02-1731/2018,1 квартал 2019
4,1-1682040,0,1 квартал 2019
...,...,...,...
25270,1-2372371217,NaN,3 квартал 2022
25271,1-2520892928,NaN,3 квартал 2022
25272,1-2669863837,NaN,3 квартал 2022
25273,1-2405188592,817303-АК/СМР-13,3 квартал 2022


In [22]:
# PART 1
# присоединяю учтенные по номеру по сибель
data = pd.merge(data, earliers_doned,  on ='Номер по SIEBEL', how ='left')

# переписываю значения в колонке Учтен
data['Учтен'] = data['Квартал учета в факте']

# после того как переписал, удаляю смерженные столбцы - из-за наличия в исходной выгрузки колонки Договор, 
# она становится Договор_x, а присоединный столбец из выгрузки учтенных - Договор_y
data.drop(columns=['Квартал учета в факте', 'Договор_y'], inplace=True)

# и возвращаю корректное название колонке Договор
data.rename(columns={'Договор_x' : 'Договор'}, inplace=True)

# смотрим что получилось 
data['Учтен'].value_counts()

основное дело у ЮД    1
3 квартал 2019        1
3 квартал 2022        1
Name: Учтен, dtype: int64

In [23]:
#PART 2
# осталось присоединить по договору, чтобы это сделать удалю в исходном списке учтенных пустые строки в колонке Договор
nan_list_1 = earliers_doned[pd.isnull(earliers_doned['Договор'])].index
earliers_doned_2 = earliers_doned.drop (index=nan_list_1)

#присоеднияю по Договору 
data = pd.merge(data, earliers_doned_2,  on ='Договор', how ='left')

data['Квартал учета в факте'].value_counts()

3 квартал 2021    5
4 квартал 2020    2
3 квартал 2019    2
2 квартал 2019    1
4 квартал 2019    1
Name: Квартал учета в факте, dtype: int64

Первая смерженная часть инф. об учете ПА была выше перенесена в поле Учет и после второго мержа у нас снова появилась колонка Квартал учета в факте, откуда будем переносить данные в колонку Учет.

__Заполнение полей через цикл и индексы здесь использую для разнообразия (в дальнейшем внизу буду работать с методом .apply() )__

In [24]:
# делаю список нанов чтобы оставить только строки с заполненным значением учета в Факте, чтобы взять индексы строк и переписать
nan_list_2 = data[pd.isnull(data['Квартал учета в факте'])].index                                                  # в цикле
hidden_data = data.drop(index=nan_list_2)
hidden_data.index

Int64Index([155, 295, 324, 325, 575, 584, 853, 866, 867, 1222, 1229], dtype='int64')

In [25]:
for indx in hidden_data.index:
    data.loc[indx, 'Учтен'] = data.loc[indx, 'Квартал учета в факте']

In [26]:
# проверяем
data['Учтен'].value_counts()

3 квартал 2021        5
4 квартал 2020        2
3 квартал 2019        2
основное дело у ЮД    1
2 квартал 2019        1
3 квартал 2022        1
4 квартал 2019        1
Name: Учтен, dtype: int64

In [27]:
# смерженные столбцы не нужны - удаляем
data.drop(columns=['Квартал учета в факте', 'Номер по SIEBEL_y'], inplace=True)

# и возвращаем корректные названия колонкам 
data.rename(columns={'Номер по SIEBEL_x' : 'Номер по SIEBEL'}, inplace=True)

## Подтягиваем активности НСТ

In [28]:
activities.columns

Index(['Мероприятие', 'Индикатор', 'Тип', 'Тип активности Outbound', 'Статус',
       'Начало', 'Фактическое окончание', 'Результат', 'Соотношение к ПЗ',
       'Завершена с просрочкой', 'Клиент (ФЛ)', 'КМБ', 'Комментарии',
       '№ договора', 'Судебное дело', 'Адрес',
       'Дней просрочки на момент создания', 'Ответственный по клиенту',
       'Центр сбора', 'Номер дела о банкротстве', 'Ответственный',
       'Дата создания', 'Дата передачи', 'Общий комментарий', 'Описание',
       'Иная судебная работа', 'Дата публичных торгов (принятие на баланс)'],
      dtype='object')

In [29]:
# в выгрузке активностей нужно всего два столбца - возьму их и переименую колонки в корректное,
# т.к. мержит надо будет по номеру по Сибель. Здесь в колонке судебное дело содержится Номер по Сибель

activities = activities.loc[:, ['Судебное дело', 'Статус']]
activities.rename(columns={'Судебное дело' : 'Номер по SIEBEL', 'Статус': 'Статус активности нст'}, inplace=True)

In [30]:
activities.head(2)

,Номер по SIEBEL,Статус активности нст
0,1-2237097491,Запланировано
1,1-531879843,Запланировано


In [31]:
# переименую значения в 1 вид (так проще перезаписывать потом)
activities['Статус активности нст'].replace("Запланировано", "Завершено", inplace=True) 
activities['Статус активности нст'].value_counts()

# отмененные не будем метить как нестандртные - они ошибочны

Завершено    265
Отменено      15
Name: Статус активности нст, dtype: int64

In [32]:
# отмененные отсеку, посколько по некоторым делам имеется несколько активностей и если в таком деле будет отмененная, то 
# она подтянется и мы не поймем что дело на самом деле нестандартной, т.к. ее учитывать не будем

activities = activities.loc[activities['Статус активности нст'] == 'Завершено']
activities['Статус активности нст'].value_counts()

Завершено    265
Name: Статус активности нст, dtype: int64

In [33]:
data = pd.merge(data, activities,  on ='Номер по SIEBEL', how ='left')
data['Статус активности нст'].value_counts()

Завершено    84
Name: Статус активности нст, dtype: int64

In [34]:
# исходя из наличия корректных активностей, заполняем поле СТ/НСТ
data['СТ/НСТ'] = np.where(data['Статус активности нст'] == 'Завершено' , 'НСТ', 'СТ')
data['СТ/НСТ'].value_counts()

СТ     1184
НСТ      84
Name: СТ/НСТ, dtype: int64

__Корректирую суммы:__

In [35]:
# у наших колонок тип данных объект, но если брать отдельные значения, то будет строковые данные:

data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268 entries, 0 to 1267
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Сумма заявленных требований      1224 non-null   object
 1   Сумма взыскания по решению суда  1268 non-null   object
dtypes: object(2)
memory usage: 29.7+ KB


In [36]:
list_to_replace = ['Сумма заявленных требований', 'Сумма взыскания по решению суда']


def replace_and_fix_values(x):
    if type(x) == str: # обхожу ошибку типа данных, когда встречается ноль
        
        # обрабатываем Руб.
        x_fixed = x.replace("Руб.", "")
        
        # приводим в корректный вид для преобразования в числовой формат
        x_fixed = x_fixed.replace(" ", "")    # пробелы между цифрами нужно также убрать
        x_fixed = x_fixed.replace(",", ".")   # в питоне у флоата точка вместо запятой
        
        # сразу убираю символ доллара и перевожу доллары в рубли
        if '$' in x_fixed:
            x_fixed = x_fixed.replace("$", "")
            x_fixed = str(float(x_fixed) * dollar_exchange_rate)  
        
        # аналогичное для евро
        if '€' in x_fixed:
            x_fixed = x_fixed.replace("€", "")
            x_fixed = str(float(x_fixed) * euro_exchange_rate)  
        
        return x_fixed
    return x

for i in list_to_replace:
    data[i] = data[i].apply(lambda x: replace_and_fix_values(x))

In [37]:
# смотрим результат
data.loc[:, list_to_replace].head(-4)

,Сумма заявленных требований,Сумма взыскания по решению суда
0,120106.50,0
1,222163.15,0
2,384892.89,0
3,59007.70,1970.23
4,51214.36,51214.36
...,...,...
1259,396364.24,0
1260,346356.65,0
1261,555649.17,555649.17
1262,150980.99,0


In [38]:
# переписываю тип данных
data = data.astype({'Сумма заявленных требований': np.float64, 'Сумма взыскания по решению суда': np.float64})

data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268 entries, 0 to 1267
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Сумма заявленных требований      1224 non-null   float64
 1   Сумма взыскания по решению суда  1268 non-null   float64
dtypes: float64(2)
memory usage: 29.7 KB


In [39]:
data.loc[:,['Сумма заявленных требований', 'Сумма взыскания по решению суда']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1268 entries, 0 to 1267
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Сумма заявленных требований      1224 non-null   float64
 1   Сумма взыскания по решению суда  1268 non-null   float64
dtypes: float64(2)
memory usage: 29.7 KB


In [40]:
# суммы отображаются корректно
data.loc[:, list_to_replace].head(3)

,Сумма заявленных требований,Сумма взыскания по решению суда
0,120106.50,0.0
1,222163.15,0.0
2,384892.89,0.0


__Смотрим что в поле Валюта__:

In [41]:
data['Валюта заявленных требований'].value_counts()

RUB    1222
USD       2
Name: Валюта заявленных требований, dtype: int64

In [42]:
data.loc[data['Валюта заявленных требований'] == 'USD', ['Общая сумма ПЗ по ОД', 'Сумма просроченных процентов']]

,Общая сумма ПЗ по ОД,Сумма просроченных процентов
11,2631.79,0.0
12,2631.79,0.0


__Меняем валютные значения у колонок:__

In [43]:
data.loc[data['Валюта заявленных требований']=='USD', 'Общая сумма ПЗ по ОД'] *= 70
data.loc[data['Валюта заявленных требований']=='USD', 'Сумма просроченных процентов'] *= 70

data.loc[data['Валюта заявленных требований']=='EUR', 'Общая сумма ПЗ по ОД'] *= 70
data.loc[data['Валюта заявленных требований']=='EUR', 'Сумма просроченных процентов'] *= 70

# сравниваем как прошел перевод - строкой выше цифры до преобразования
data.loc[data['Валюта заявленных требований'] == 'USD', ['Общая сумма ПЗ по ОД', 'Сумма просроченных процентов']]

,Общая сумма ПЗ по ОД,Сумма просроченных процентов
11,184225.3,0.0
12,184225.3,0.0


Тип сокр и РОО

##  Переписываю поле Тип

Суть в том, что Банкроты могут быть МБ или РБ, но Банкротов РБ МБ нужно учесть как МБ

In [44]:
data['Тип сокр'] = data['Тип']

In [45]:
data['Тип сокр'] = data['Тип сокр'].convert_dtypes()

In [46]:
data['Тип сокр'].value_counts()

Банкротство ФЛ РБ                861
Взыскание задолженности ПА РБ    344
Взыскание задолженности ПА МБ     45
Банкротство ЮЛ МБ                 13
Банкротство ФЛ МБ                  5
Name: Тип сокр, dtype: Int64

In [47]:
data['Тип сокр'] = np.where((data['Тип'] == 'Банкротство ФЛ РБ'), 'Банкротство РБ', data['Тип сокр'])
data['Тип сокр'] = np.where((data['Тип'] == 'Банкротство ЮЛ МБ'), 'Банкротство МБ', data['Тип сокр'])    
data['Тип сокр'] = np.where((data['Тип'] == 'Банкротство ФЛ МБ'), 'Банкротство МБ', data['Тип сокр'])    
data['Тип сокр'] = np.where((data['Тип'] == 'Взыскание задолженности ПА РБ'), 'Взыскание РБ', data['Тип сокр'])
data['Тип сокр'] = np.where((data['Тип'] == 'Взыскание задолженности ПА МБ'), 'Взыскание МБ', data['Тип сокр'])


In [48]:
# смотрим результат
data.groupby(['Тип сокр', 'Тип']).agg({'Номер по SIEBEL': 'count'})

# все корректно

Номер по SIEBEL
Тип сокр       Тип                                           
Банкротство МБ Банкротство ФЛ МБ                            5
               Банкротство ЮЛ МБ                           13
Банкротство РБ Банкротство ФЛ РБ                          861
Взыскание МБ   Взыскание задолженности ПА МБ               45
Взыскание РБ   Взыскание задолженности ПА РБ              344

## Создаем колонку дата конца:

In [49]:
data['Дата конца'] = date_of_quarter_end
data['Дата конца'].value_counts()

2023-03-31    1268
Name: Дата конца, dtype: int64

In [50]:
data.loc[:, ['Дата конца', ]]

,Дата конца
0,2023-03-31
1,2023-03-31
2,2023-03-31
3,2023-03-31
4,2023-03-31
...,...
1263,2023-03-31
1264,2023-03-31
1265,2023-03-31
1266,2023-03-31


__Добавляю колонки, значения по умолчанию будут None, чтобы потом можно было себя перепроверить__

In [51]:
columns_to_add = ['Кол-во дней', 'Коэф', 'Факт', 'План']

for i in columns_to_add:
    data[i] = None

# Заполняем колонки:

__Колонка Кол-во дней:__

Здесь есть особенность по делам созданным либо завершенным в период с 15 дек по 19 янв - по ним срок увеличен на 35 дней. Это нужно учитывать

In [52]:
# промежуточный вариант для понимания что будет выводиться
d = data.loc[data.index == 5]['Дата конца'] - data.loc[data.index == 5]['Дата создания']
d

5   171 days
dtype: timedelta64[ns]

In [53]:
# так будем забирать только число
d.dt.days

5    171
dtype: int64

In [54]:
# ф-ция расчета кол-ва дней  - по завершенным вычитается Дата закрытия из Даты создания, а по активным из Даты конца.

def count_num_days(row):
#     try:
    if row['Статус'] == 'Завершено':
        if (row['Дата создания'] >= datetime(2022, 12, 15, 0, 0, 0, 1).date()) and \
        row['Дата создания'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date():
            date_x = row['Дата закрытия'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days

        if (row['Дата закрытия'] >= datetime(2022, 12, 31, 23, 59, 0, 1).date()) and \
        (row['Дата закрытия'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date()):
            date_x = row['Дата закрытия'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days

        date_x = row['Дата закрытия'] - row['Дата создания']
        return date_x.days

    # пошел блок по не завершенным делам
    else:
        if (row['Дата создания'] >= datetime(2022, 12, 15, 0, 0, 0, 1).date()) and \
        (row['Дата создания'] <= datetime(2023, 1, 19, 0, 0, 0, 1).date()):
            date_x = row['Дата конца'] - row['Дата создания'] - timedelta(days=35)
            return date_x.days

        # для всех остальных случаев
        date_x = row['Дата конца'] - row['Дата создания']
        return date_x.days
#     except TypeError:
#         row
data['Кол-во дней'] = data.apply(count_num_days, axis=1)

In [55]:
#  РАБОТАЕТ, ВЫКЛЮЧИЛ ЧТОБЫ НАПИСАТЬ опмтимальнее - см. выше

# indx_closed_deals = data.loc[data['Статус'] == 'Завершено'].index
# indx_active_deals = data.loc[data['Статус'] != 'Завершено'].index

# for indx in indx_closed_deals:
#     data.loc[indx, 'Кол-во дней'] = data.loc[indx, 'Дата закрытия'] - data.loc[indx, 'Дата создания']
    
# for indx in indx_active_deals:
#     data.loc[indx, 'Кол-во дней'] = data.loc[indx, 'Дата конца'] - data.loc[indx, 'Дата создания']
    
# # перевожу в числовой формат, чтобы можно было потом сравнивать
# data['Кол-во дней'] = data['Кол-во дней'].apply(lambda x: x.days)

__Сокращу кол-во колонок для удобства написания функции расчета коэффициента:__

In [56]:
cols_list = ['Номер по SIEBEL', 'РОО', 'Статус', 'Сумма заявленных требований','Сумма взыскания по решению суда',\
             'Дата создания', 'Дата конца','Комментарии','Дата закрытия','Учтен','Кол-во дней', 'Коэф','СТ/НСТ', \
             'Статус активности нст','Тип сокр','Общая сумма ПЗ по ОД','Сумма просроченных процентов','Факт', 'План']

#выключить при демонстрации
# list_for_work = ['Ответственный','Договор', 'Заемщик ФЛ', 'Заемщик КМБ']
# cols_list = cols_list + list_for_work
        
data_cuted = data.loc[:, cols_list]
data_cuted.head(3)

,Номер по SIEBEL,РОО,Статус,Сумма заявленных требований,Сумма взыскания по решению суда,Дата создания,Дата конца,Комментарии,Дата закрытия,Учтен,Кол-во дней,Коэф,СТ/НСТ,Статус активности нст,Тип сокр,Общая сумма ПЗ по ОД,Сумма просроченных процентов,Факт,План
0,1-2922685695,Нальчик,В работе,120106.50,0.0,2022-10-11,2023-03-31,Иск.Наследники. Иск направлен в суд - 19.10.20...,NaT,NaN,171,None,СТ,NaN,Взыскание РБ,59281.32,7139.14,None,None
1,1-2566500297,Нальчик,В работе,222163.15,0.0,2022-06-06,2023-03-31,Иск. Наследники. Исковое заявление напр. в суд...,NaT,NaN,298,None,СТ,NaN,Взыскание РБ,175467.50,46695.65,None,None
2,1-2566858573,Нальчик,В работе,384892.89,0.0,2022-06-06,2023-03-31,Иск.Наследники. Исковое заявление напр. в суд ...,NaT,NaN,298,None,СТ,NaN,Взыскание РБ,315937.21,47946.16,None,None


In [57]:
# выведу типы дел, чтобы под них написать ф-цию расчета коэффициента
data_cuted['Тип сокр'].value_counts()

Банкротство РБ    861
Взыскание РБ      344
Взыскание МБ       45
Банкротство МБ     18
Name: Тип сокр, dtype: int64

In [58]:
# Коэф. одинаковые, но у стандартных дел свои сроки, у нест. - свои.
def get_coef(row):
    if row['Статус'] == 'Завершено':
        if row['СТ/НСТ'] == 'СТ':                     # Блок Завершенные СТ
            if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] <= 60:
                    return 1.5
                if (row['Кол-во дней'] > 60) and (row['Кол-во дней'] <= 100):
                    return 1
                if (row['Кол-во дней'] > 100) and (row['Кол-во дней'] <= 220):
                    return 0.3
                if row['Кол-во дней'] > 220:
                    return 0.7
                
            if row['Тип сокр'] == 'Банкротство МБ':
                if row['Кол-во дней'] <= 80:
                    return 1.5
                if (row['Кол-во дней'] > 80) and (row['Кол-во дней'] <= 170):
                    return 1
                if (row['Кол-во дней'] > 170) and (row['Кол-во дней'] <= 290):
                    return 0.3
                if row['Кол-во дней'] > 290:
                    return 0.7
                
            if row['Тип сокр'] == 'Взыскание МБ':
                if row['Кол-во дней'] <= 160:
                    return 1.5
                if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 180):
                    return 1
                if (row['Кол-во дней'] > 180) and (row['Кол-во дней'] <= 300):
                    return 0.3
                if row['Кол-во дней'] > 300:
                    return 0.7
                
            if row['Тип сокр'] == 'Взыскание РБ':
                if row['Кол-во дней'] <= 160:
                    return 1.5
                if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 170):
                    return 1
                if (row['Кол-во дней'] > 170) and (row['Кол-во дней'] <= 290):
                    return 0.3
                if row['Кол-во дней'] > 290:
                    return 0.7
                
        # Блок Завершенные НСТ
        if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] <= 60:
                    return 1.5
                if (row['Кол-во дней'] > 60) and (row['Кол-во дней'] <= 250):
                    return 1
                if (row['Кол-во дней'] > 250) and (row['Кол-во дней'] <= 370):
                    return 0.3
                if row['Кол-во дней'] > 370:
                    return 0.7
                
        if row['Тип сокр'] == 'Банкротство МБ':
            if row['Кол-во дней'] <= 80:
                return 1.5
            if (row['Кол-во дней'] > 80) and (row['Кол-во дней'] <= 390):
                return 1
            if (row['Кол-во дней'] > 390) and (row['Кол-во дней'] <= 510):
                return 0.3
            if row['Кол-во дней'] > 510:
                return 0.7

        if row['Тип сокр'] == 'Взыскание МБ':
            if row['Кол-во дней'] <= 180:
                return 1.5
            if (row['Кол-во дней'] > 180) and (row['Кол-во дней'] <= 300):
                return 1
            if (row['Кол-во дней'] > 300) and (row['Кол-во дней'] <= 420):
                return 0.3
            if row['Кол-во дней'] > 420:
                return 0.7

        if row['Тип сокр'] == 'Взыскание РБ':
            if row['Кол-во дней'] <= 160:
                return 1.5
            if (row['Кол-во дней'] > 160) and (row['Кол-во дней'] <= 270):
                return 1
            if (row['Кол-во дней'] > 270) and (row['Кол-во дней'] <= 390):
                return 0.3
            if row['Кол-во дней'] > 390:
                return 0.7
                
                
    # пошел блок активных дел (не завершенные)
    else:
        if row['СТ/НСТ'] == 'СТ':                       # активные СТ
            if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] < 100:
                    return 0
                if row['Кол-во дней'] > 100:
                    if (row['Кол-во дней'] >= 190) and (row['Кол-во дней'] <= 220):      
                        return 0
                    return 1
                    
                
            if row['Тип сокр'] == 'Банкротство МБ':
                if row['Кол-во дней'] < 170:
                    return 0
                if row['Кол-во дней'] > 170:
                    if (row['Кол-во дней'] >= 260) and (row['Кол-во дней'] <= 290):       
                        return 0
                    return 1
                
            if row['Тип сокр'] == 'Взыскание МБ':
                if row['Кол-во дней'] < 180:
                    return 0
                if row['Кол-во дней'] > 180:
                    if (row['Кол-во дней'] >= 270) and (row['Кол-во дней'] <= 300):       
                        return 0
                    return 1
                
            if row['Тип сокр'] == 'Взыскание РБ':
                if row['Кол-во дней'] < 170:
                    return 0
                if row['Кол-во дней'] > 170:
                    if (row['Кол-во дней'] >= 260) and (row['Кол-во дней'] <= 290):      
                        return 0
                    return 1
                
        # пошел блок активных НСТ
        if row['Тип сокр'] == 'Банкротство РБ':
                if row['Кол-во дней'] < 250:
                    return 0
                if row['Кол-во дней'] >= 250:
                    if (row['Кол-во дней'] >= 340) and (row['Кол-во дней'] <= 370):       
                        return 0
                    return 1   
                
        if row['Тип сокр'] == 'Банкротство МБ':
            if row['Кол-во дней'] < 390:
                return 0
            if row['Кол-во дней'] >= 390:
                if (row['Кол-во дней'] >= 480) and (row['Кол-во дней'] <= 510):       
                    return 0
                return 1

        if row['Тип сокр'] == 'Взыскание МБ':
            if row['Кол-во дней'] < 300:
                return 0
            if row['Кол-во дней'] >= 300:
                if (row['Кол-во дней'] >= 390) and (row['Кол-во дней'] <= 420):       
                    return 0
                return 1

        if row['Тип сокр'] == 'Взыскание РБ':
            if row['Кол-во дней'] < 270:
                return 0
            if row['Кол-во дней'] >= 270:
                if (row['Кол-во дней'] >= 360) and (row['Кол-во дней'] <= 390):      
                    return 0
                return 1
            
data_cuted['Коэф'] = data_cuted.apply(get_coef, axis=1)

__Помним что ранее учтенные дела не подлежат учету в СМОТ текущего квартала.__

Обнуляем коэф. по этим делам:

In [59]:
data_cuted['Учтен'] = data_cuted['Учтен'].convert_dtypes()

In [60]:
data_cuted['Учтен'].value_counts()

3 квартал 2021        7
4 квартал 2020        4
основное дело у ЮД    3
3 квартал 2019        2
2 квартал 2019        1
3 квартал 2022        1
4 квартал 2019        1
Name: Учтен, dtype: Int64

In [61]:
# ф-ция обнуления коэф. по учтеным ранее делам
def check_early_doned(row):
    if type(row['Учтен']) == str:   # проверяю по типу данных, т.к. если не заполнено, то будет 0, а это цифра
        return 0
    return row['Коэф']
               
data_cuted['Коэф'] = data_cuted.apply(check_early_doned, axis=1)

In [62]:
# проверям, должно быть 0 в коэф и наличие инф. об учете в поле Учтен
data_cuted.loc[data_cuted['Учтен'] == '3 квартал 2021']

,Номер по SIEBEL,РОО,Статус,Сумма заявленных требований,Сумма взыскания по решению суда,Дата создания,Дата конца,Комментарии,Дата закрытия,Учтен,Кол-во дней,Коэф,СТ/НСТ,Статус активности нст,Тип сокр,Общая сумма ПЗ по ОД,Сумма просроченных процентов,Факт,План
296,1-2921632422,Краснодар,В работе,8176283.11,1486753.05,2022-10-10,2023-03-31,"Подан иск по трем к/д, дело не назначено на 25...",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
325,1-2921632433,Краснодар,В работе,17888861.65,0.00,2022-10-10,2023-03-31,"Иск подан по трем КД, дело не назначено на 25....",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
326,1-2921632445,Краснодар,В работе,3675434.56,0.00,2022-10-10,2023-03-31,"Иск подан по трем КД, дело не назначено на 25....",NaT,3 квартал 2021,172,0.0,СТ,NaN,Взыскание МБ,0.00,0.0,None,None
883,1-2138028794,Краснодар,В работе,2247091.81,2247091.81,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,6270554.37,711908.4,None,None
884,1-2138028794,Краснодар,В работе,2247091.81,2247091.81,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,6270554.37,711908.4,None,None
885,1-2138028806,Краснодар,В работе,635509.83,635509.83,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,1520059.55,194033.9,None,None
886,1-2138028806,Краснодар,В работе,635509.83,635509.83,2021-12-28,2023-03-31,"иск направлен 10.01 с/з 09.02, 09.03 возражени...",2022-08-10,3 квартал 2021,458,0.0,НСТ,Завершено,Взыскание МБ,1520059.55,194033.9,None,None


In [63]:
# в колонке Коэф. все корректно
data_cuted['Коэф'].value_counts()

0.0    683
1.0    350
1.5    179
0.3     55
0.7      1
Name: Коэф, dtype: int64

__Осталось занулить коэф по делам о только обращении взыскания на залог после ИНН:__

Это тоже особенность СМОТирования. <br>
В поле Комментарии содержатся в т.ч. Маркеры, которые заполняются ответственными, и по делам об обращении взыскания на залог после ИНН содержится маркер "обращение на залог после ИНН".

In [64]:
data_cuted['Комментарии'] = data_cuted['Комментарии'].convert_dtypes()
data_cuted['Комментарии'][0:3]

0    Иск.Наследники. Иск направлен в суд - 19.10.20...
1    Иск. Наследники. Исковое заявление напр. в суд...
2    Иск.Наследники. Исковое заявление напр. в суд ...
Name: Комментарии, dtype: string

In [65]:
# ищу вхождение части маркера в поле Комментарии
search_to = 'залог после'

indx_serched = data_cuted[data_cuted["Комментарии"].str.contains(search_to)].index

for indx in indx_serched:
    data_cuted.loc[indx, 'Коэф'] = 0

In [66]:
data_cuted['Коэф'].value_counts()

0.0    732
1.0    327
1.5    156
0.3     52
0.7      1
Name: Коэф, dtype: int64

Видим изменения в кол-ве дел по коэффициентам.

Переходим в финальной части

## Вычисляем факт и план:

In [67]:
def compute_fact(row):
    if row['Статус'] == 'Завершено':
        sum_in_fact = row['Коэф'] * row['Сумма взыскания по решению суда']
        return sum_in_fact

def compute_plan(row):
    if row['Статус'] == 'Завершено':
        
        if (row['Коэф'] == 1) or(row['Коэф'] == 0.7):
            
            # по банкротам сразу заявл. требования будем тянуть
            if (row['Тип сокр'] == 'Банкротство РБ') or (row['Тип сокр'] == 'Банкротство МБ'):
                return row['Сумма заявленных требований']
            
            # по не банкротам суммируем ОД и %
            result = row['Общая сумма ПЗ по ОД'] + row['Сумма просроченных процентов']
            if result == 0:
                result = row['Сумма заявленных требований']
            
            return result
    
    # блок по делам не завершенным
    else:
        if row['Коэф'] == 1:
            if (row['Тип сокр'] == 'Банкротство РБ') or (row['Тип сокр'] == 'Банкротство МБ'):
                return row['Сумма заявленных требований']
            
            # по не банкротам
            result = row['Общая сумма ПЗ по ОД'] + row['Сумма просроченных процентов']
            if result == 0:
                result = row['Сумма заявленных требований']
            
            return result

data_cuted['План'] = data_cuted.apply(compute_plan, axis=1)
data_cuted['Факт'] = data_cuted.apply(compute_fact, axis=1)

In [68]:
# РАЗОБРАТЬСЯ ПОЗЖЕ ОТКУДА ПОЯВЛЯЮТСЯ ДУБЛИКАТЫ 
data_cuted.drop_duplicates(keep='first',inplace=True)

## Формируем таблицу  result_table:

In [69]:
plan = pd.pivot_table(data_cuted, values='План', index=['РОО'], columns=['Тип сокр'], aggfunc=np.sum)

fact = pd.pivot_table(data_cuted, values='Факт', index=['РОО'], columns=['Тип сокр'], aggfunc=np.sum)

In [70]:
plan

Тип сокр,Банкротство МБ,Банкротство РБ,Взыскание МБ,Взыскание РБ
РОО,,,,
Волгоград,0.00,28366471.75,200314.08,4518624.30
Краснодар,97149862.23,53478509.02,6519691.10,15141801.18
Нальчик,1890182.18,21865480.47,NaN,4316070.26
Ростов,0.00,77662151.06,15105167.72,4280286.53
Ставрополь,NaN,5326736.08,3524417.50,1662527.41


In [71]:
fact

Тип сокр,Банкротство МБ,Банкротство РБ,Взыскание МБ,Взыскание РБ
РОО,,,,
Волгоград,0.000,3.675471e+07,1999883.148,9733471.269
Краснодар,0.000,5.143539e+07,510070.470,1038740.892
Нальчик,0.000,4.113376e+07,NaN,4105890.042
Ростов,1976396.325,1.235736e+08,808145.705,187443.669
Ставрополь,NaN,4.900104e+06,0.000,3830543.040


In [72]:
for i in plan.columns:
    plan.rename(columns={i : i + ' план'}, inplace=True)

plan.columns

Index(['Банкротство МБ план', 'Банкротство РБ план', 'Взыскание МБ план',
       'Взыскание РБ план'],
      dtype='object', name='Тип сокр')

In [73]:
for i in fact.columns:
    fact.rename(columns={i : i + ' факт'}, inplace=True)

fact.columns

Index(['Банкротство МБ факт', 'Банкротство РБ факт', 'Взыскание МБ факт',
       'Взыскание РБ факт'],
      dtype='object', name='Тип сокр')

In [74]:
fact = pd.merge(fact, plan,  on ='РОО', how ='left')
fact.head(2)

Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план
РОО,,,,,,,,
Волгоград,0.0,3.675471e+07,1999883.148,9733471.269,0.00,28366471.75,200314.08,4518624.30
Краснодар,0.0,5.143539e+07,510070.470,1038740.892,97149862.23,53478509.02,6519691.10,15141801.18


In [75]:
fact.fillna(0, inplace=True) 

In [76]:
def compute_percent(row, col_name):
     
    # буду передавать название Тип сокр
    if row[f'{col_name} план'] == 0:
        return 100.00
    
    result = row[f'{col_name} факт'] / row[f'{col_name} план'] * 100
    return float(f'{result: .2f}')


fact['% Банкротство РБ'] = fact.apply(compute_percent, axis=1, col_name='Банкротство РБ')
fact['% Банкротство МБ'] = fact.apply(compute_percent, axis=1, col_name='Банкротство МБ')
fact['% Взыскание РБ'] = fact.apply(compute_percent, axis=1, col_name='Взыскание РБ')
fact['% Взыскание МБ'] = fact.apply(compute_percent, axis=1, col_name='Взыскание МБ')
#fact.fillna(0, inplace=True) 
fact



Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план,% Банкротство РБ,% Банкротство МБ,% Взыскание РБ,% Взыскание МБ
РОО,,,,,,,,,,,,
Волгоград,0.000,3.675471e+07,1999883.148,9733471.269,0.00,28366471.75,200314.08,4518624.30,129.57,100.0,215.41,998.37
Краснодар,0.000,5.143539e+07,510070.470,1038740.892,97149862.23,53478509.02,6519691.10,15141801.18,96.18,0.0,6.86,7.82
Нальчик,0.000,4.113376e+07,0.000,4105890.042,1890182.18,21865480.47,0.00,4316070.26,188.12,0.0,95.13,100.00
Ростов,1976396.325,1.235736e+08,808145.705,187443.669,0.00,77662151.06,15105167.72,4280286.53,159.12,100.0,4.38,5.35
Ставрополь,0.000,4.900104e+06,0.000,3830543.040,0.00,5326736.08,3524417.50,1662527.41,91.99,100.0,230.40,0.00


### Переводим числовое отображение в читаемый вид:

In [77]:
a = [i for i in fact.columns if 'факт' in i]
b = [i for i in fact.columns if 'план' in i]
a = a + b
a

['Банкротство МБ факт',
 'Банкротство РБ факт',
 'Взыскание МБ факт',
 'Взыскание РБ факт',
 'Банкротство МБ план',
 'Банкротство РБ план',
 'Взыскание МБ план',
 'Взыскание РБ план']

In [78]:
import decimal

def make_money_view(some_float):
    n = decimal.Decimal(f'{some_float: .2f}')  #(f'{some_float: .2f}')    # n = decimal.Decimal(str(some_float))
    return '{0:,}'.format(n).replace(',', ' ')

for i in a:
    fact[i] = fact[i].apply(lambda x: make_money_view(x))
    
fact


Тип сокр,Банкротство МБ факт,Банкротство РБ факт,Взыскание МБ факт,Взыскание РБ факт,Банкротство МБ план,Банкротство РБ план,Взыскание МБ план,Взыскание РБ план,% Банкротство РБ,% Банкротство МБ,% Взыскание РБ,% Взыскание МБ
РОО,,,,,,,,,,,,
Волгоград,0.00,36 754 714.31,1 999 883.15,9 733 471.27,0.00,28 366 471.75,200 314.08,4 518 624.30,129.57,100.0,215.41,998.37
Краснодар,0.00,51 435 394.24,510 070.47,1 038 740.89,97 149 862.23,53 478 509.02,6 519 691.10,15 141 801.18,96.18,0.0,6.86,7.82
Нальчик,0.00,41 133 757.93,0.00,4 105 890.04,1 890 182.18,21 865 480.47,0.00,4 316 070.26,188.12,0.0,95.13,100.00
Ростов,1 976 396.32,123 573 628.21,808 145.71,187 443.67,0.00,77 662 151.06,15 105 167.72,4 280 286.53,159.12,100.0,4.38,5.35
Ставрополь,0.00,4 900 104.46,0.00,3 830 543.04,0.00,5 326 736.08,3 524 417.50,1 662 527.41,91.99,100.0,230.40,0.00


### Располагаем столбцы в логической последовательности:

In [79]:
result_table = fact[['Банкротство МБ факт', 'Банкротство МБ план', '% Банкротство МБ',\
              'Банкротство РБ факт', 'Банкротство РБ план', '% Банкротство РБ',\
              'Взыскание МБ факт', 'Взыскание МБ план', '% Взыскание МБ',\
              'Взыскание РБ факт', 'Взыскание РБ план', '% Взыскание РБ'
        ]]

   

result_table

Тип сокр,Банкротство МБ факт,Банкротство МБ план,% Банкротство МБ,Банкротство РБ факт,Банкротство РБ план,% Банкротство РБ,Взыскание МБ факт,Взыскание МБ план,% Взыскание МБ,Взыскание РБ факт,Взыскание РБ план,% Взыскание РБ
РОО,,,,,,,,,,,,
Волгоград,0.00,0.00,100.0,36 754 714.31,28 366 471.75,129.57,1 999 883.15,200 314.08,998.37,9 733 471.27,4 518 624.30,215.41
Краснодар,0.00,97 149 862.23,0.0,51 435 394.24,53 478 509.02,96.18,510 070.47,6 519 691.10,7.82,1 038 740.89,15 141 801.18,6.86
Нальчик,0.00,1 890 182.18,0.0,41 133 757.93,21 865 480.47,188.12,0.00,0.00,100.00,4 105 890.04,4 316 070.26,95.13
Ростов,1 976 396.32,0.00,100.0,123 573 628.21,77 662 151.06,159.12,808 145.71,15 105 167.72,5.35,187 443.67,4 280 286.53,4.38
Ставрополь,0.00,0.00,100.0,4 900 104.46,5 326 736.08,91.99,0.00,3 524 417.50,0.00,3 830 543.04,1 662 527.41,230.40


__Сохраняю таблицу в эксель:__

In [81]:
writer = pd.ExcelWriter(name_for_table) 
result_table.to_excel(writer) 
writer.save()

__Сохраняю преобразованный датафрейм с эксель:__

In [80]:
writer = pd.ExcelWriter(name_for_file_output)
data_cuted.to_excel(writer) 
writer.save()
